In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython
%reload_ext Cython

In [2]:
import sys
sys.path.append('..')
import warnings
def warn(message = None, category=None, stacklevel=1, source=None):
    pass

warnings.warn = warn

In [3]:
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
#custom
from data_functions import make_date_to_MYR
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )

from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

In [4]:
%%time
pp_str = 'Pre_Matching_DATA/'

treated2 = pd.read_pickle(pp_str+'treated2.p')
controls = pd.read_pickle(pp_str+'controls.p')
pmpm =pd.read_pickle(pp_str+'pmpm.p')
#random_controls = pd.read_pickle('tempHybridSelectionMoreData/random.p')

CPU times: user 7.04 s, sys: 14.8 s, total: 21.8 s
Wall time: 53.6 s


In [5]:
pmpm.head(2)

PERS_ID     MYR MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT MEDICAID_LOB  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  \
0      -3  201908        M              119          0   Commercial       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
1      -3  201909        M              119          0   Commercial       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   

   CC_OSTEOPOROSIS  CC_PERIPHERAL  CC_PERSONALITY  CC_PRESSURE  CC_PROSTATE_CANCER  CC_PTSD  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  LOB  PREGNANCY  IP_BIN  ER_BIN  ACUTE  IP_C  ER_C  ACUTE_C  IP_BIN6  ER_BIN6  ACUTE6  IP_BIN2  ER_BIN2  ACUTE2  
0              0.0            0.0             0.0          0.0                 0.0      0.0               0.0                     0.0             0.0              0.0        0.0         0.0                 0.0               0.0    1        0.0       0       0      0     0     0        0      0.0      0.0     0.0      0.0      0.0     0.0  
1              0.0            0.0             0.0          0.0                 0.0      0.0               0.0                     0.0             0.0              0.0        0.0         0.0                 0.0               0.0    1        0.0       0       0      0     0     0        0      0.0      0.0     0.0      0.0      0

### For each month find the month corresponding to 6 months back

In [6]:
pgroups = pmpm[['PERS_ID', 'MYR']]
pgroups['day'] = 1
pgroups['month'] = pgroups.MYR%100
pgroups['year'] = pgroups.MYR//100
pgroups['MYR2'] = pd.to_datetime(pgroups[['day', 'month', 'year']])
pgroups['MYRPAST'] = pgroups['MYR2']-pd.DateOffset(months = 6)
pgroups['MYR3'] = pd.DatetimeIndex(pgroups['MYRPAST']).year*100+pd.DatetimeIndex(pgroups['MYRPAST']).month
pgroups['HasPast'] = 0
pgroups = pgroups[['PERS_ID', 'MYR', 'MYR3','HasPast']]

In [7]:
pgroups.head(5)

PERS_ID     MYR    MYR3  HasPast
0      -3  201908  201902        0
1      -3  201909  201903        0
2      -3  201910  201904        0
3      -3  201911  201905        0
4      -3  201912  201906        0

### Groupby by ids and transform groups into list of groups for speedup

In [8]:
%%time
groups = pgroups.groupby('PERS_ID')
lst = list(groups)

CPU times: user 13.6 s, sys: 787 ms, total: 14.4 s
Wall time: 14.4 s


In [9]:
%%cython

def  timecount(long[:] flags, long[:] MYR, long[:] MYR2):
    
    cdef int i
    cdef int N = len(flags)
    
    for i in range(6,N):
        if MYR[i-6] == MYR2[i]:
            flags[i] = 1
            
    return 0

### Calculate the column HasPast which is 1 if the month in the time point 6 months back is the same as the 6th month on the past

In [10]:
%%time
for i,group in lst:
    x = group
    myr = np.array(x['MYR'])
    date = np.array(x['MYR3'])
    flags = np.array(x['HasPast'])
    timecount(flags, myr, date)
    group['HasPast'] = flags


CPU times: user 1min 8s, sys: 126 ms, total: 1min 8s
Wall time: 1min 8s


### Concatenate all the dataframes together

In [11]:
%%time
lst2 = [l[1] for l in lst]
finaldat = pd.concat(lst2, ignore_index = True)

CPU times: user 43.2 s, sys: 1.11 s, total: 44.3 s
Wall time: 43.5 s


In [12]:
finaldat.head(5)

PERS_ID     MYR    MYR3  HasPast
0      -3  201908  201902        0
1      -3  201909  201903        0
2      -3  201910  201904        0
3      -3  201911  201905        0
4      -3  201912  201906        0

### Put the has past column in the initial Pmpm

In [13]:
pmpm['HasPast'] = finaldat['HasPast'].values

In [14]:
finaldat2 = finaldat[['PERS_ID', 'MYR', 'HasPast']]

In [15]:
treated2.shape, controls.shape, pmpm.shape

### Put the HasPast column in the controls and treated

In [16]:
treated2 = pd.merge(treated2, finaldat2, on = ['PERS_ID', 'MYR'])
controls = pd.merge(controls, finaldat2, on = ['PERS_ID', 'MYR'])

In [49]:
pp_str = 'Pre_Matching_DATA/'

treated2.to_pickle(pp_str+'treated2Past.p')
controls.to_pickle(pp_str+'controlsPast.p')

In [17]:
treated2.shape, controls.shape, pmpm.shape

#### Hierarchical Clustering
> Divide the Treated population in two clusters and study the <strong>Onset of Diabetes Onset</strong> separately in each of the clusters

In [18]:
unique = len(treated2.PERS_ID.unique())
clusters = int(unique**(1/2))
unique
clusters = 2

In [19]:
columns_to_match_hier = ['AGE_AT_MIDMONTH', 'ALLOW_AMT', 'MBR_GNDR','CC_TOBACCO', 'CC_PRESSURE', 'CC_OBESITY',
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID', 'CC_DX_CNT']

treated0, treated1, model, link_matr,  clustermapdat, treated22 = agglomerative(treated2.copy(),
                                                                                columns_to_match_hier,
                                                                                n_clusters = clusters,
                                                                               filterpast = True)

In [20]:
treated22 = treated22.reset_index(drop = True)

In [50]:
treated22.shape

In [21]:
treated22.head(4)

PERS_ID     MYR  MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT MEDICAID_LOB  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  \
0  2602482  201908         0               81 -23.025851   Commercial       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  1.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                1          1                    0.0          0.0              0.0          0.0                1.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
1  2603072  202001         0               82   4.615121     Medicare       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  1.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          3                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                1.0              1.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
2  2606370  202010         0               82   3.135494     Medicare       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  1.0           0.0        0.0            1.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            1.0                      0.0                0.0          0.0                0          5                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         1.0   
3  2608221 

In [22]:
treated3 = pd.merge(treated2, treated22[['PERS_ID', 'Group']],  on = 'PERS_ID')

In [57]:
treated3.PERS_ID.unique().shape

In [61]:
treated3.shape,treated2.shape

#### Visualization of the clusters created in the previous step with a clustermap

In [23]:
g = sns.clustermap(clustermapdat, method = 'ward', standard_scale=1, row_linkage = link_matr, col_cluster = False)

#### For each of the two clusters formed from the Treated patients form a matched population from the controls.
<div style="background:Aliceblue"> We use hier matching as our matching function</div>

In [24]:
controls2  = controls[controls['HasPast'] == 1].reset_index(drop = True)

In [25]:
controls2.shape, controls.shape

In [26]:
#cols = list(col for col in treated2.columns if col not in ['PERS_ID', 'MYR'])
#columns_to_match = cols
columns_to_match =  ['AGE_AT_MIDMONTH', 'ALLOW_AMT', 'MBR_GNDR','CC_TOBACCO', 'CC_PRESSURE', 'CC_OBESITY', 
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID', 'CC_DX_CNT', 'ACUTE2', 'ACUTE6', 'IP_BIN6', 'ER_BIN6','LOB']

In [29]:
%%time

#columns_to_match = columns_to_match + ['IP_BIN2', 'ER_BIN2']
#hier_cols = ['IP_BIN2', 'ER_BIN2', 'ACUTE2', 'ACUTE6', 'IP_BIN6', 'ER_BIN6']
match_on = ['AGE_AT_MIDMONTH', 'ALLOW_AMT', 'CC_TOBACCO', 'CC_PRESSURE', 'CC_OBESITY', 
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID', 'CC_DX_CNT', 'ACUTE2']
hier_cols = ['IP_BIN6', 'ER_BIN6']
hier_range = [1,1]
exact_match_dict = {'IP_BIN6':[0,1,2], 'ER_BIN6':[0,1,2], 'LOB':None}
nnr_func_args = {'match_on':match_on, 'hier_cols':hier_cols,
                        'hier_range':hier_range, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': True, 'exact_match_dict': exact_match_dict}


matched1, matched_data1 = matched_population(treated3.copy(), 
                                           controls2.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)

Mathcing Function: nearest_neighbor_func
hERE
After filtering control ids are:269017 before they were:299160
IP_BIN6 [0, 1, 2]
ER_BIN6 [0, 1, 2]
LOB None
There were no matches for 0 out of 1367 patients
CPU times: user 32min 36s, sys: 20min 36s, total: 53min 12s
Wall time: 53min 12s


### Save data

In [30]:
Path('./tempNNhierAgglotheory').mkdir(exist_ok = True)
p = './tempNNhierAgglotheory/'
matched_data1.to_pickle(p+'match1.p')
treated1.to_pickle(p+'treated1.p')



In [31]:
with open(p+'matchesPast.p', 'wb') as handle:
    pickle.dump(matched1, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [32]:
pmpm2 = pmpm
try:
    pmpm2.MBR_GNDR.replace(to_replace = {'F':0, 'M':1}, inplace = True)
except:
    pass

In [33]:
from utils import statistics_first_month, p_val, stat_matrices 

### List containing all numerical columns

In [34]:
all_columns = list(set(col for col in pmpm.columns)-set(['PERS_ID', 'MYR','MEDICAID_LOB']))

In [35]:
treat_test = treated22.copy()
treat_test.set_index('PERS_ID', inplace = True)
unique_groups = sorted(list(treat_test.Group.unique()))
len(unique_groups)

### Initialize groups

In [36]:
groups = {}
groups_id = {}
for group in unique_groups:
    groups[group] = pd.DataFrame()
    groups_id[group] = []

### Get For each group treated ids and matched controls

In [37]:
for (key, month), val in matched1.items():
    gt = treat_test.loc[key].Group
    val['RD'] = val.MYR
    groups[gt] = pd.concat((groups[gt], val), ignore_index = True)
    groups_id[gt].append(key)

In [38]:
treat_test.loc['2602482'].Group

1

In [48]:
len(groups_id[0]), len(groups_id[1]), groups[0].shape, groups[1].shape

In [52]:
720+624,3592+3225

### Get the treated data groups

In [58]:
from collections import defaultdict

treated_groups = {}
for group in groups:
    ids = groups_id[group]
    
    treated_groups[group] = treated3[treated3.PERS_ID.isin(ids)].reset_index(drop = True)

In [59]:
len(treated_groups[0].PERS_ID.unique()), len(treated_groups[1].PERS_ID.unique())

### Get the whole controls data

In [62]:
%%time
control_groups = {}
for group in groups:
    dat = groups[group]
    dat2 = dat[['PERS_ID', 'RD']]
    dat2 = dat2.drop_duplicates(subset = ['PERS_ID'], ignore_index = True)
    new_dat = pd.merge(controls, dat2, on = 'PERS_ID')
    filt = new_dat.MYR >= new_dat.RD
    new_dat2 = new_dat[filt].reset_index(drop = True)
    control_groups[group] = new_dat2
    

CPU times: user 2.06 s, sys: 479 ms, total: 2.54 s
Wall time: 2.54 s


In [64]:
len(control_groups[0].PERS_ID.unique()), len(control_groups[1].PERS_ID.unique())

In [65]:
treated_groups[0].shape, control_groups[0].shape,treated_groups[1].shape, control_groups[1].shape

In [66]:
with open(p+'cgpast.p', 'wb') as handle:
    pickle.dump(control_groups, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(p+'tgpast.p', 'wb') as handle:
    pickle.dump(treated_groups, handle, protocol=pickle.HIGHEST_PROTOCOL)